In [14]:
import h2o
from h2o.estimators import H2OStackedEnsembleEstimator, H2ORandomForestEstimator, H2OGradientBoostingEstimator

### Explore a abordagem de Stacking através do treino de um modelo de Stacked Ensemble utilizando os modelos treinados nos exercícios anteriores.

#### 1. Utilizando Python pra realizar o stacking dos modelos treinados anteriormente. Recorde a importância de replicar os dados de treino e inclua o keep_cross_validations_predictions=TRUE durante o treino dos seus modelos DRF e GBM.

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,30 mins 49 secs
H2O_cluster_timezone:,Europe/Lisbon
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,19 days
H2O_cluster_name:,H2O_from_python_Alice_Dias_d0aocp
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.854 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [ ]:
# Carregamento dos dados
train = h2o.import_file("train.csv")
test = h2o.import_file("test.csv")

# Remover colunas irrelevantes
train = train.drop(["C1", "id"])
test = test.drop(["C1", "id"])

# Definir target e features
target = "satisfaction"
features = [col for col in train.columns if col != target]

# Modelo DRF com validação cruzada
drf_model = H2ORandomForestEstimator(
    ntrees=100,
    nfolds=5,
    fold_assignment="Modulo",
    keep_cross_validation_predictions=True,
    seed=42
)
drf_model.train(x=features, y=target, training_frame=train)

In [ ]:
# Modelo GBM com validação cruzada
gbm_model = H2OGradientBoostingEstimator(
    ntrees=100,
    max_depth=5,
    learn_rate=0.1,
    nfolds=5,
    fold_assignment="Modulo",
    keep_cross_validation_predictions=True,
    seed=42
)

# Treinar o modelo
gbm_model.train(x=features, y=target, training_frame=train)

In [16]:
# IDs dos melhores modelos treinados (confirma o nome dos objetos!)
base_models = [drf_model.model_id, gbm_model.model_id]

# Criar e treinar o Stacked Ensemble
ensemble = H2OStackedEnsembleEstimator(
    base_models=base_models,
    metalearner_algorithm="AUTO",
    seed=42
)
ensemble.train(x=features, y=target, training_frame=train)

# Avaliação de performance
ensemble_performance = ensemble.model_performance()
print(ensemble_performance)


stackedensemble Model Build progress: |██████████████████████████████████████████| (done) 100%
ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.01049583685757379
RMSE: 0.10244919159063086
LogLoss: 0.045739236042022284
AUC: 0.9993037056732302
AUCPR: 0.9991111625207248
Gini: 0.9986074113464605
Null degrees of freedom: 9930
Residual degrees of freedom: 9928
Null deviance: 13562.079094965142
Residual deviance: 908.4727062666466
AIC: 914.4727062666466

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.38957817264551553
                         neutral or dissatisfied    satisfied    Error    Rate
-----------------------  -------------------------  -----------  -------  --------------
neutral or dissatisfied  5615                       65           0.0114   (65.0/5680.0)
satisfied                63                         4188         0.0148   (63.0/4251.0)
Total                    5678                       4253         0.0129   (128.0/9931.0)

Maximum Met

##### a) Analise os resultados de performance. Foi capaz de obter mlhrias?
O modelo Stacked Ensemble, treinado c os modelos DRF e GBM cm base, obteve um desempenho superior em tds as principais métricas:

- maior AUC (0.9993), > F1 (0.9850) e > precisão geral
- < LogLoss (0.0457) → previsões confiantes e robustas
- A redução da taxa de erro para 1.29% confirma a eficácia do ensemble

✅ S, foi possível obter mlhrias claras face aos modelos individuais. O ensemble combinou as forças do DRF e GBM, resultando num classificador ainda + poderoso e generalizável.